<span><h1 style="color:#4987FF">
Speed-Optimisation for Scientific Python
</h1></span>

# Target Audience:

### People with -
* Basic working knowledge of python and common maths/science libraries (numpy, scipy)

* A functioning Python3 installation. (Python 2 should also be fine but may require minor changes to some parts of the code)

* Ability to install the following modules/libraries

> "pip install timeit"

> "pip install cython" <- not needed with Anaconda

> "pip install py-heat-magic"

> "pip install line_profiler"

> "pip install numba"

* (preferably) basic knowledge of Jupyter Notebooks. (Some slight differences in some techniques if replicated on the REPL or in a script)

In [1]:
%load_ext Cython
%load_ext line_profiler
%load_ext heat
import timeit
from functools import lru_cache as cache
from numba import jit, autojit, int64
import numpy as np
from numpy import linalg as LA

<span><h1 style="color:#4987FF">
Example 2: An inefficient fibonacci generator
</h1></span>
### First a look at the base case; what we will be trying to improve

In [2]:
def regular_fib(n):
    if n<2:
        return n
    return regular_fib(n-1)+regular_fib(n-2)

### Now we see if compiling it in C and using static typing improves the speed

In [3]:
%%cython

cpdef long cython_fib(long n):
    if n<2:
        return n
    return cython_fib(n-1)+cython_fib(n-2)

### Here we try using 'cacheing / memoization' - remembering the results of previous evaluations. $\therefore$ fib($n$) will only be calculated once $\forall n \leq 30$

In [4]:
@cache(maxsize=None)
def cacheing_fib(n):
    if n<2:
        return n
    return cacheing_fib(n-1)+cacheing_fib(n-2)

### Finally, we try using Numba's just-in-time compiler - this attempts to convert array / complex mathematical expressions into optimised machine code

In [5]:
@autojit
def jit_fib(n):
    if n<2:
        return n
    return jit_fib(n-1)+jit_fib(n-2)

## Results:

In [6]:
%timeit -n1 -r1 regular_fib(30)
print('\n\n')
%timeit -n1 -r1 jit_fib(30)
print('\n\n')
%timeit -n1 -r1 cython_fib(30)
print('\n\n')
%timeit -n1 -r1 cacheing_fib(30)

1 loop, best of 1: 1.57 s per loop



1 loop, best of 1: 1.29 s per loop



1 loop, best of 1: 12.6 ms per loop



1 loop, best of 1: 75.5 µs per loop


## Comments:


In [7]:
# TODO: re-evaluate the times
print("jit improvement ~ {:,.0f}%".format(round(100*(1.74 / 175e-3),-2)))
print("cython improvement ~ {:,.0f}%".format(round(100*(1.74 / 12.3e-3),-2)))
print("cacheing improvement ~ {:,.0f}%".format(round(100*(1.74 / 72.3e-6),-4)))

jit improvement ~ 1,000%
cython improvement ~ 14,100%
cacheing improvement ~ 2,410,000%


In [8]:
%lprun -f regular_fib regular_fib(30)
# Just showing the line profiler being used

In [9]:
# Here's my attempt at a maximally optimised version, mixes in some mathematical optimisation with computational optimisation
# This isn't what this talk is focusing on, just showing that you don't need to choose between one method or the other

@cache(maxsize=None)
def luke_fib(n):
    if (n<2):
        return n
    if n%2!=0:
        return luke_fib((n+1)//2)**2 + luke_fib((n-1)//2)**2
    half = luke_fib(n//2)
    return half**2 + 2*half*luke_fib(n//2 -1)

In [26]:
%timeit -n1 -r1 luke_fib(1000)

1 loop, best of 1: 173 µs per loop


In [11]:
%%cython

# Here we check if using a non-recursive method would have helped much

cpdef long long non_recursive_fib(long n):
    a,b = 0,1
    for i in range(n-2):
        a,b = b,b+a
    return b

In [20]:
%timeit -n1 -r1 non_recursive_fib(1000)

1 loop, best of 1: 783 µs per loop


Exception ignored in: '_cython_magic_b59f27d2215fefd63ec9f4a8d04c6d50.non_recursive_fib'
OverflowError: int too big to convert


In [16]:
# That seems pretty fast, but what is that 'OverflowError' talking about?

non_recursive_fib(1000)

Exception ignored in: '_cython_magic_b59f27d2215fefd63ec9f4a8d04c6d50.non_recursive_fib'
OverflowError: int too big to convert


0

In [17]:
# We told cython to expect to output a 'long long' which is it's largest default type for an integer; so it allocated memory 
# for this output at the start. 
# However, the 1000th fibonacci number is far too big to fit into even this large pre-allocated memory, and so gave the above
# OverflowError. 
# This is one place python has an advantage over strongly typed languages; it can allocate more memory as it 
# needs to, letting us evaluate the 1000th fibonacci number without complaining; 

luke_fib(1000)

43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875